<a href="https://colab.research.google.com/github/robin9804/Openholo_Val/blob/master/Depth_map_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# Depthmap 기반 홀로그램 생성

fresnel integral 기반 back propagation

In [ ]:
# parameters
mm = 10e-3
um = mm*mm
nm = um*mm
wvl_R = 639 * nm  # Red
wvl_G = 525 * nm  # Green
wvl_B = 463 * nm  # Blue

def delta(wvl):
    return wvl * 10  # sampling period

def k0(wvl):
    return (np.pi * 2) / wvl  # wave number

pp = 3.45 * um  # pixel to pixel parameter
z_scale = 0.3  # depth range to real scale

# resolution setting
width = 1920 
height = 1080

# Depthmap file read

depth map bmp file을 읽어오면서 깊이 정보를 가져온다.

In [ ]:
# depth map 조건
field_len = 1000e-3
near_depth = 800e-3
far_depth = 1200e-3

# depth quantization (깊이 계조)
DQ = 256  # 0 to 255

# unit depth 
UD = (far_depth - near_depth) / 256

# file path
depthfile = 'file path'
rgbfile = 'file path'

depthmap = Image.open(depthfile)
depthmap = np.array(depthmap)
rgbmap = Image.open(rgbfile)
rgbmap = np.array(rgbmap)


# Back propagation 을 사용하여 hologram 취득


In [ ]:
# alpha map extract
def alphamap(n):
    """
    extract alpha map
    """
    amap = np.zeros((width, height))
    for i in range(width):
        for j in range(height):
            if depthmap[i,j] == n:
                amap[i,j] = 1
    return amap

def h_Fresnel(x1, y1, z1, x2, y2, z2, wvl):
    """
    Fresnel integral inpulse response
    """
    z = abs(z2 - z1)
    x = x1 - x2
    y = y1 - y2
    a = np.exp(1j* k0(wvl) * z)
    h = a * np.exp(1j*(k0(wvl)/(2*z))*(x**2 + y**2))
    return h

def T(n, wvl):
    """
    plaen to plane propagation operator
    n : order
    """
    z = UD * n
    g1 = np.zeros((height, width))
    for i in range(width):
        for j in range(height):
            if depthmap[j,i] == n:
                g1[j,i] = rgbmap[j,i,1] * np.exp(1j*k0(wvl) * z)
    return g1

def depth_prop(wvl):
    i = 0
    plane = np.zeros((height, width))
    while i < DQ:
        if i == 0:
            plane = T(0, wvl)
        else:
            plane = T(i, wvl) * alphamap(i) + plane * (1 - alphamap(i))
        print(i," th plane done")
        i +=1
    return plane

# Back propagation을 통한 depth map hologram 계산

In [ ]:
def depth_prop(wvl):
    i = 0
    plane = np.zeros((height, width))
    while i < DQ:
        if i == 0:
            plane = T(0, wvl)
        else:
            plane = T(i, wvl) * alphamap(i) + plane * (1 - alphamap(i))
        print(i," th plane done")
        i +=1
    return plane

# 병렬 계산을 통한 depth map hologram 생성

In [ ]:
def depth_parallel(wvl):
    h = np.zeros((height, width))
    for n in range(DQ):
        h = h + T(n, wvl)
    return h
